In [51]:
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import sklearn

matplotlib.use("Qt5Agg")

In [53]:
def preprocess(df):
    df['width'] = df['circuit'].apply(lambda x: int(x.split('-')[0].strip('q')))
    df['depth'] = df['circuit'].apply(lambda x: int(x.split('-')[1].split('.')[0]))
    df.drop(columns=['circuit'], inplace=True)
    return df

stl = preprocess(pd.read_csv("linux_single_results.csv"))
mtl = preprocess(pd.read_csv("linux_mthread_results.csv"))

In [55]:
stl.describe()

,contraction_ns,contracted_exe_ns,normal_exe_ns,width,depth
count,1.500000e+02,1.500000e+02,1.500000e+02,150.000000,150.000000
mean,9.822120e+06,1.132362e+09,2.458162e+09,6.000000,255.000000
std,6.400489e+06,1.652421e+09,3.671339e+09,0.819232,144.792144
min,1.984740e+05,1.066288e+06,2.103459e+06,5.000000,10.000000
25%,4.681217e+06,1.408855e+08,1.030199e+08,5.000000,130.000000
50%,9.277949e+06,4.922842e+08,6.424230e+08,6.000000,255.000000
75%,1.372803e+07,1.241295e+09,3.427950e+09,7.000000,380.000000
max,2.783427e+07,7.562061e+09,1.322357e+10,7.000000,500.000000


In [56]:
from sklearn.linear_model import LinearRegression

def linreg(x, y):
    reg = LinearRegression().fit(x.values.reshape(-1, 1), y.values)
    xnew = np.linspace(0, max(x), 20)
    ynew = reg.coef_ * xnew + reg.intercept_
    ynew[ynew < 0] = 0
    return xnew, ynew

def polyreg(x, y, degree):
    reg = np.polyfit(x, y, degree)
    xnew = np.linspace(0, max(x), 20)
    ynew = np.polyval(reg, xnew)
    return xnew, ynew

## Depth vs Contraction Time

In [59]:
x5 = stl[stl['width'] == 5]['depth']
x6 = stl[stl['width'] == 6]['depth']
x7 = stl[stl['width'] == 7]['depth']
y5 = stl[stl['width'] == 5]['contraction_ns'] / 1e6
y6 = stl[stl['width'] == 6]['contraction_ns'] / 1e6
y7 = stl[stl['width'] == 7]['contraction_ns'] / 1e6

# use sklearn to make a linear regression
x5new, y5new = linreg(x5, y5)
x6new, y6new = linreg(x6, y6)
x7new, y7new = linreg(x7, y7)

fig, ax = plt.subplots()

# give a title
ax.set_title('Contraction time vs circuit depth')
ax.set_xlabel('Circuit depth')
ax.set_ylabel('Contraction time (ms)')

ax.plot(x5, y5, 'x:', label='5 qubits', color='#1f77b4')
ax.plot(x6, y6, 'x:', label='6 qubits', color='#ff7f0e')
ax.plot(x7, y7, 'x:', label='7 qubits', color='#2ca02c')
ax.plot(x5new, y5new, label='5 qubits linear regression', color='#1f77b4')
ax.plot(x6new, y6new, label='6 qubits linear regression', color='#ff7f0e')
ax.plot(x7new, y7new, label='7 qubits linear regression', color='#2ca02c')

plt.legend()
plt.grid()
plt.show()

# save fig
fig.savefig('depth_vs_contraction_time.png', dpi=300)

## Depth vs Execution Time

In [60]:
f = lambda s: stl[stl['width'] == s]

x5 = f(5)['depth']
x6 = f(6)['depth']
x7 = f(7)['depth']
y5c = f(5)['contracted_exe_ns'] / 1e9
y5n = f(5)['normal_exe_ns'] / 1e9
y6c = f(6)['contracted_exe_ns'] / 1e9
y6n = f(6)['normal_exe_ns'] / 1e9
y7c = f(7)['contracted_exe_ns'] / 1e9
y7n = f(7)['normal_exe_ns'] / 1e9

# use sklearn to make a linear regression
x5cnew, y5cnew = linreg(x5, y5c)
x5nnew, y5nnew = linreg(x5, y5n)
x6cnew, y6cnew = linreg(x6, y6c)
x6nnew, y6nnew = linreg(x6, y6n)
x7cnew, y7cnew = linreg(x7, y7c)
x7nnew, y7nnew = linreg(x7, y7n)

fig, ax = plt.subplots()

# give a title
ax.set_title('Execution time vs circuit depth')
ax.set_xlabel('Circuit depth')
ax.set_ylabel('Execution time (s)')

ax.plot(x5, y5c, 'x:', label='Contraction on 5q', color='#42c2f5')
ax.plot(x5, y5n, 'x:', label='Normal on 5q', color='#4287f5')
ax.plot(x6, y6c, 'x:', label='Contraction on 6q', color='#88f289')
ax.plot(x6, y6n, 'x:', label='Normal on 6q', color='#34e036')
ax.plot(x7, y7c, 'x:', label='Contraction on 7q', color='#ff7a38')
ax.plot(x7, y7n, 'x:', label='Normal on 7q', color='#de5d3a')
ax.plot(x5cnew, y5cnew, color='#42c2f5')
ax.plot(x5nnew, y5nnew, color='#4287f5')
ax.plot(x6cnew, y6cnew, color='#88f289')
ax.plot(x6nnew, y6nnew, color='#34e036')
ax.plot(x7cnew, y7cnew, color='#ff7a38')
ax.plot(x7nnew, y7nnew, color='#de5d3a')

plt.legend()
plt.grid()
plt.show()

# save fig
fig.savefig('depth_vs_execution_time.png', dpi=300)

## Depth vs (Execution Time difference)

In [61]:
x5 = f(5)['depth']
x6 = f(6)['depth']
x7 = f(7)['depth']
y5 = f(5)['normal_exe_ns'] / 1e9 - f(5)['contracted_exe_ns'] / 1e9
y6 = f(6)['normal_exe_ns'] / 1e9 - f(6)['contracted_exe_ns'] / 1e9
y7 = f(7)['normal_exe_ns'] / 1e9 - f(7)['contracted_exe_ns'] / 1e9

# use sklearn to make a linear regression
x5new, y5new = polyreg(x5, y5, 2)
x6new, y6new = polyreg(x6, y6, 2)
x7new, y7new = polyreg(x7, y7, 2)

fig, ax = plt.subplots()

# give a title
ax.set_title('Time gained vs circuit depth')
ax.set_xlabel('Circuit depth')
ax.set_ylabel('Time gained (s)')

ax.plot(x5, y5, 'x:', label='5 qubits', color='#1f77b4')
ax.plot(x6, y6, 'x:', label='6 qubits', color='#ff7f0e')
ax.plot(x7, y7, 'x:', label='7 qubits', color='#2ca02c')
ax.plot(x5new, y5new, label='5 qubits linear regression', color='#1f77b4')
ax.plot(x6new, y6new, label='6 qubits linear regression', color='#ff7f0e')
ax.plot(x7new, y7new, label='7 qubits linear regression', color='#2ca02c')

plt.legend()
plt.grid()
plt.show()

# save fig
fig.savefig('depth_vs_time_gained.png', dpi=300)